In [1]:
from nariflow import start_initializer

start_initializer().initializer('tape')

In [70]:
from nariflow import Variable
from nariflow import optimizer
from nariflow import GradientTape
#from nariflow import calc_gradient
from nariflow import layer
from nariflow.models import Model
from nariflow import functions as f
from nariflow.core import elementary_function as ef
from nariflow.core import shape_function as sf
from nariflow.utils import DataSet
import numpy as np
import pandas as pd

np.set_printoptions(suppress=True)

import tensorflow as tf

import matplotlib.pyplot as plt

import time

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from tensorflow.keras.layers.experimental import preprocessing

In [3]:
import tensorflow as tf

from tensorflow.keras import datasets, layers, models

In [4]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
test_images = test_images.reshape((10000, 28, 28, 1))

# 픽셀 값을 0~1 사이로 정규화합니다.
train_images, test_images = train_images / 255.0, test_images / 255.0

In [56]:
train_images = train_images.reshape(train_images.shape[0],
                                     train_images.shape[3],
                                     train_images.shape[1],
                                    train_images.shape[2])

test_images = test_images.reshape(test_images.shape[0],
                                     test_images.shape[3],
                                     test_images.shape[1],
                                    test_images.shape[2])

In [48]:
class Models(Model):
    def __init__(self):
        super().__init__()
        self.l1 = layer.Conv2d(64, (3,3))
        self.l2 = layer.Linear(10)
        self.flatten = layer.flatten

    def forward(self, x):
        y = self.l1(x)
        y = f.activation.relu(y)
        y = self.flatten(y)
        y = self.l2(y)

        return y

lr = 0.01

model = Models()

optimizers = optimizer.Adam(lr)

optimizers.setup(model)

In [49]:
def train_step(X_train, y_train):
    with GradientTape() as tape:
        y_pred = model(X_train)
        loss = f.loss.softmax_cross_entropy(y_pred, y_train)

    tape.CalcGradient()
    optimizers.update()

    if j % 100 == 0:
        print(loss.data)

In [50]:
iters = 0

batch_size = 20

start_time = time.time()

In [51]:
ind = np.random.choice(np.arange(len(train_labels)), 5000, replace = False)

In [52]:
while True:
    for j in range(np.ceil((len(ind) / batch_size)).astype('int')):

        train = train_images[ind][j * batch_size : (j + 1) * batch_size]
        label = train_labels[ind][j * batch_size : (j + 1) * batch_size]

        train_step(train, label)

    print('total_time : ', time.time() - start_time)

    iters += 1

    print(iters)
    
    if iters > 2:
        break

2.303135472063416
0.44070487939001823
0.11843639737940179
total_time :  28.102504014968872
1
0.10944934940470444
0.04109966395019325
0.01274266207804956
total_time :  61.42779994010925
2
0.01077948831698321
0.011232295485630659
0.0023454080777399523
total_time :  99.93848276138306
3


In [55]:
test_images

(10000, 28, 28, 1)

In [68]:
test_pred = model(test_images)

In [75]:
print(f.loss.softmax(test_pred.data).data[0:100])

[[0.         0.         0.         0.         0.         0.
  0.         0.99999999 0.         0.        ]
 [0.         0.00000004 0.99998994 0.00000769 0.         0.
  0.00000233 0.         0.         0.        ]
 [0.000005   0.9998913  0.0000107  0.00000205 0.00001261 0.00000026
  0.0000033  0.00000018 0.00007454 0.00000005]
 [0.99999982 0.         0.00000015 0.         0.         0.
  0.00000002 0.         0.         0.        ]
 [0.         0.         0.00000003 0.00000002 0.99967407 0.
  0.00000012 0.0000003  0.00000013 0.00032532]
 [0.00001154 0.99993565 0.00000248 0.00000144 0.00000661 0.00000001
  0.         0.00000126 0.00003576 0.00000526]
 [0.         0.         0.00000823 0.00000015 0.99997886 0.
  0.         0.00000001 0.00001208 0.00000067]
 [0.         0.00004471 0.00001972 0.00000987 0.00058901 0.00095468
  0.00000001 0.00004926 0.00001914 0.9983136 ]
 [0.         0.         0.         0.         0.00000007 0.00236783
  0.99711798 0.         0.00051412 0.        ]
 [0.0

In [85]:
print('test셋 정확도 : ', accuracy_score(np.argmax(test_pred.data, axis = -1), test_labels))

test셋 정확도 :  0.9456


In [86]:
print('혼동행렬')
confusion_matrix(np.argmax(test_pred.data, axis = -1), test_labels)

혼동행렬


array([[ 961,    0,    3,    4,    1,    6,   17,    0,   11,    7],
       [   0, 1104,    8,    0,    1,    1,    2,    6,    4,    7],
       [   3,    8,  938,    5,    5,    3,    1,   15,    6,    2],
       [   0,    2,   14,  952,    0,   16,    0,    3,   10,    4],
       [   2,    1,    9,    1,  910,    2,    7,    5,    2,    9],
       [   0,    1,    1,    9,    4,  825,   12,    1,    5,    8],
       [   9,    4,    2,    0,    9,   10,  914,    0,    9,    0],
       [   3,    1,   16,   10,   10,    2,    0,  989,    4,   13],
       [   1,   14,   40,   24,    9,   25,    5,    4,  915,   11],
       [   1,    0,    1,    5,   33,    2,    0,    5,    8,  948]],
      dtype=int64)